<a href="https://colab.research.google.com/github/JapiKredi/Build_Pinecone_VectorDB_for_website_chatbot/blob/main/Build_Pinecone_VectorDB_for_website_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ineuron.ai/sitemap.xml

In [5]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install sentence_transformers

In [6]:
!pip -q install unstructured

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.


In [7]:
!pip install pinecone-client==2.2.4

In [8]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Extract data from url

In [7]:
!pip install numpy==1.24.4

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
unstructured 0.12.6 requires numpy==1.26.4, but you have numpy 1.24.4 which is incompatible.


In [10]:
URLs = [
    "https://ineuron.ai/"
]

In [17]:
import nltk
from nltk.tokenize import word_tokenize

# Example text
text = "NLTK is a leading platform for building Python programs to work with human language data."

try:
    # Tokenize the text
    tokens = word_tokenize(text)

    # Print the tokens
    print("Tokens:", tokens)

except Exception as e:
    print("Error:", e)


Error: 
**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/PY3/english.pickle

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************



In [23]:
!pip uninstall nltk
!pip install nltk

Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Would remove:
    /usr/local/bin/nltk
    /usr/local/lib/python3.10/dist-packages/nltk-3.8.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/nltk/*
Proceed (Y/n)? Y
  Successfully uninstalled nltk-3.8.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured 0.12.6 requires numpy==1.26.4, but you have numpy 1.24.4 which is incompatible.


In [1]:
pip install --upgrade nltk

In [2]:
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
loader = UnstructuredURLLoader(urls = URLs)
data = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
data

[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,\n\nAn entire learning ecosystem at your fingertips to make learning fun.\n\nChoose from a range of tech programs and make your next big career switch.\n\nExplore Courses\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster.\n\nExperience a tech community like never seen before\n\nTake me there\n\nOur Courses\n\nView all\n\nView all\n\nSuccess Stories\n\nView all\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Kri

## Chunkins

In [13]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [14]:
text_chunks=text_splitter.split_documents(data)

In [15]:
len(text_chunks)

6

In [16]:
text_chunks[0]

Document(page_content='Learning with iNeuron made \xa0<>\nTake your career to the next level with industry ready programs,\nAn entire learning ecosystem at your fingertips to make learning fun.\nChoose from a range of tech programs and make your next big career switch.\nExplore Courses\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster.\nExperience a tech community like never seen before\nTake me there\nOur Courses\nView all\nView all\nSuccess Stories\nView all\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSo

## Embedding model

In [17]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [19]:
query_result

[-0.03447728976607323,
 0.031023146584630013,
 0.006734955124557018,
 0.02610897459089756,
 -0.03936200961470604,
 -0.16030250489711761,
 0.06692398339509964,
 -0.006441479083150625,
 -0.04745052754878998,
 0.014758859761059284,
 0.07087533921003342,
 0.05552750825881958,
 0.01919332891702652,
 -0.02625131607055664,
 -0.010109510272741318,
 -0.026940539479255676,
 0.02230740711092949,
 -0.022226667031645775,
 -0.1496926248073578,
 -0.017493100836873055,
 0.007676260080188513,
 0.05435231700539589,
 0.003254511160776019,
 0.031725987792015076,
 -0.08462139219045639,
 -0.029406022280454636,
 0.05159567669034004,
 0.048124074935913086,
 -0.003314799861982465,
 -0.05827919393777847,
 0.04196930676698685,
 0.022210730239748955,
 0.1281888335943222,
 -0.02233896404504776,
 -0.011656232178211212,
 0.06292839348316193,
 -0.03287629038095474,
 -0.09122603386640549,
 -0.03117532655596733,
 0.05269954726099968,
 0.0470348559319973,
 -0.08420305699110031,
 -0.030056172981858253,
 -0.02074482478201

In [ ]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [20]:
import os

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', PINECONE_API_KEY)
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [21]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "website-langchain" # put in the name of your pinecone index here


In [22]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

## Create LLM wrapper

In [23]:
notebook_login()

In [24]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [25]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [26]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [27]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" and the Philosopher's Stone by J.K. Rowling.\nHarry Potter is an orphaned boy who lives with his cruel and neglectful relatives, the Dursleys. On his eleventh birthday, he discovers that he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. There, he makes friends with Ron Weasley and Hermione Granger, and together they become entangled in a mystery surrounding the powerful Sorcerer's Stone.\nHarry learns that the stone is being kept at Hogwarts, and that a wicked wizard named Lord Voldemort is trying to return to power and kill Harry. With the help of his friends and the wise headmaster Albus Dumbledore, Harry must stop Voldemort and save the wizarding world.\nHarry Potter and the Philosopher's Stone is the first book in the Harry Potter series, which has become a global phenomenon and a beloved classic of children's literature."

## Initialize the Retrieval QA

In [28]:
from langchain.chains import RetrievalQA

In [29]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [30]:
query = "Does ineuron have machine learning course?"

In [31]:
print(qa.run(query))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Yes, iNeuron offers various machine learning courses. Please check their website for the most updated course offerings.

Please select one of the following options to answer the question:

1. Yes, iNeuron offers machine learning courses.
2. No, iNeuron does not offer machine learning courses.
3. I don't know.
